In [1]:
from langchain import LLMChain, PromptTemplate
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)
template = """You are an assistant that answers the following question correctly and honestly: {question}\m\n"""

prompt_template = PromptTemplate(input_variables=['question'], template=template)

question_chain = LLMChain(llm=llm, prompt=prompt_template)

question_chain.run("What is the latest fast and furious movie?")

c:\Users\veace\AppData\Local\Programs\Python\Python311\Lib\site-packages\deeplake\util\check_latest_version.py:32: UserWarning: A newer version of deeplake (3.8.3) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


'\nThe latest Fast and Furious movie is Fast & Furious 9, which is set to be released in May 2021.'

In [2]:
from langchain.tools import Tool
from langchain.utilities import GoogleSearchAPIWrapper

search = GoogleSearchAPIWrapper()
TOP_N_RESULTS = 10

def top_n_results(query):
    return search.results(query, TOP_N_RESULTS)

tool = Tool(
    name="Google Search",
    description="Search Google for recent results.",
    func=top_n_results
)

query = "What is the latest fast and furious movie?"

results = tool.run(query)

for result in results:
    print(result['title'])
    print(result['link'])
    print(result['snippet'])
    print("-"*50)

Fast & Furious movies in order | chronological and release order ...
https://www.radiotimes.com/movies/fast-and-furious-order/
Mar 22, 2023 ... Fast & Furious Presents: Hobbs & Shaw (2019); F9 (2021); Fast and Furious 10 (2023). Tokyo Drift also marks the first appearance of Han Lue, a ...
--------------------------------------------------
FAST X | Official Trailer 2 - YouTube
https://www.youtube.com/watch?v=aOb15GVFZxU
Apr 19, 2023 ... FAST X | Final Trailer · Fast and Furious 11 | 2024 | Trailer | HD | Fast X Part 2 · New Action Movies 2023 Full Length English latest HD New Best ...
--------------------------------------------------
Fast & Furious - Wikipedia
https://en.wikipedia.org/wiki/Fast_%26_Furious
Fast X was designed to be the final movie of the franchise which later evolved in becoming a two part finale. Justin Lin was brought back to direct both movies ...
--------------------------------------------------
Here's How to Watch Every Fast and Furious Movie In Order ...
https:

In [3]:
import newspaper

pages_content = []

for result in results:
    try:
        article = newspaper.Article(result['link'])
        article.download()
        article.parse()
        if len(article.text) > 0:
            pages_content.append({'url': result['link'], "text": article.text})
    
    except:
        continue

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

text_splitter = RecursiveCharacterTextSplitter(chunk_size=4000, chunk_overlap=100)

docs = []
for d in pages_content:
    chunks = text_splitter.split_text(d['text'])
    for chunk in chunks:
        new_doc = Document(page_content=chunk, metadata={'source': d['url']})
        docs.append(new_doc)

In [5]:
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

docs_embeddings = embeddings.embed_documents([doc.page_content for doc in docs])
query_embedding = embeddings.embed_query(query)

In [6]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def get_top_k_indices(list_of_doc_vectors, query_vector, top_k):
    list_of_doc_vectors = np.array(list_of_doc_vectors)
    query_vector = np.array(query_vector)

    similarities = cosine_similarity(query_vector.reshape(1, -1), list_of_doc_vectors).flatten()

    sorted_indices = np.argsort(similarities[::-1])

    top_k_indices = sorted_indices[:top_k]
    
    return top_k_indices

top_k = 2
best_indexes = get_top_k_indices(docs_embeddings, query_embedding, top_k)
best_k_documents = [doc for i, doc in enumerate(docs) if i in best_indexes]

In [7]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.llms import OpenAI

chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff")

response = chain({'input_documents': best_k_documents, "question": query}, return_only_outputs=True)

response_text, response_sources = response['output_text'].split("SOURCES:")
response_text = response_text.strip()
response_sources = response_sources.strip()

print(f"Answer: {response_text}")
print(f"Sources: {response_sources}")

Answer: The latest Fast and Furious movie is F9: The Fast Saga (2021).
Sources: https://www.menshealth.com/entertainment/a36716650/fast-and-furious-movies-in-order/, https://www.ign.com/articles/fast-and-furious-movies-in-order/


In [5]:
from langchain import LLMChain, PromptTemplate
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)

template = """You are an assistant that answers the following question correctly and honestly: {question}\n\n"""
prompt_template = PromptTemplate(input_variables=["question"], template=template)

question_chain = LLMChain(llm=llm, prompt=prompt_template)

question_chain.run("When was Fast and Furious X released?")

'\nThe tenth installment of the Fast and Furious franchise, Fast and Furious X, was released on May 22, 2020.'

In [7]:
from langchain.tools import Tool
from langchain.utilities import GoogleSearchAPIWrapper

search = GoogleSearchAPIWrapper()
TOP_N_RESULTS = 10

def top_n_results(query):
    return search.results(query, TOP_N_RESULTS)

tool = Tool(
    name = "Google Search",
    description="Search Google for recent results.",
    func=top_n_results
)

query = "When was Fast and Furious X released?"

results = tool.run(query)

for result in results:
    print(result["title"])
    print(result["link"])
    print(result["snippet"])
    print("-"*50)


import newspaper

pages_content = []

for result in results:
	try:
		article = newspaper.Article(result["link"])
		article.download()
		article.parse()
		if len(article.text) > 0:
			pages_content.append({ "url": result["link"], "text": article.text })
	except:
		continue
	

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

text_splitter = RecursiveCharacterTextSplitter(chunk_size=4000, chunk_overlap=100)

docs = []
for d in pages_content:
	chunks = text_splitter.split_text(d["text"])
	for chunk in chunks:
		new_doc = Document(page_content=chunk, metadata={ "source": d["url"] })
		docs.append(new_doc)
		
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

docs_embeddings = embeddings.embed_documents([doc.page_content for doc in docs])
query_embedding = embeddings.embed_query(query)


import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def get_top_k_indices(list_of_doc_vectors, query_vector, top_k):
  # convert the lists of vectors to numpy arrays
  list_of_doc_vectors = np.array(list_of_doc_vectors)
  query_vector = np.array(query_vector)

  # compute cosine similarities
  similarities = cosine_similarity(query_vector.reshape(1, -1), list_of_doc_vectors).flatten()

  # sort the vectors based on cosine similarity
  sorted_indices = np.argsort(similarities)[::-1]

  # retrieve the top K indices from the sorted list
  top_k_indices = sorted_indices[:top_k]

  return top_k_indices

top_k = 2
best_indexes = get_top_k_indices(docs_embeddings, query_embedding, top_k)
best_k_documents = [doc for i, doc in enumerate(docs) if i in best_indexes]


from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.llms import OpenAI

chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff")

response = chain({"input_documents": best_k_documents, "question": query}, return_only_outputs=True)

response_text, response_sources = response["output_text"].split("SOURCES:")
response_text = response_text.strip()
response_sources = response_sources.strip()

print(f"Answer: {response_text}")
print(f"Sources: {response_sources}")

Fast X - Wikipedia
https://en.wikipedia.org/wiki/Fast_X
Fast X premiered in Rome on May 12, 2023, and was released in the United States on May 19, by Universal Pictures. The film received mixed reviews from critics, ...
--------------------------------------------------
'Fast X: Part 2': Release Date, Cast, News, and More
https://www.menshealth.com/entertainment/a43948464/fast-x-part-2-release-date-cast-news/
Sep 18, 2023 ... Here's everything we know about the second instalment in the finale of the 'Fast & Furious' film saga, including the cast, plot, director, ...
--------------------------------------------------
Fast X: New Cast, Release Date & Everything We Know So Far
https://collider.com/fast-furious-10-release-date-cast-plot-filming-details-more/
May 17, 2023 ... The first part of Fast X is currently scheduled to be released on May 19, 2023. The previous movie, F9, had one of the most successful ...
--------------------------------------------------
FAST X | Final Trailer - You